In [1]:
import pandas as pd
import numpy as np
import requests

## read the JSON file that you saved in ex02

In [2]:
df = pd.read_json('../data/auto.json')

In [3]:
pd.options.display.float_format = '{:,.2f}'.format

In [4]:
df.head()

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,"3,200.00",Ford,Focus
1,E432XX77RUS,1,"6,500.00",Toyota,Camry
2,7184TT36RUS,1,"2,100.00",Ford,Focus
3,X582HE161RUS,2,"2,000.00",Ford,Focus
4,92918M178RUS,1,"5,700.00",Ford,Focus


## enrich the dataframe using a sample from that dataframe

In [5]:
df_sample = df.groupby(['CarNumber', 'Make', 'Model']).first().reset_index()
df_sample = df_sample.drop_duplicates() ### надо???
df_sample = df_sample.sample(n=200, random_state=21).reset_index(drop=True)

In [6]:
df_sample

,CarNumber,Make,Model,Refund,Fines
0,O64097197RUS,Ford,Focus,2,"8,594.59"
1,97998K152RUS,Ford,Focus,2,"6,300.00"
2,T941CC96RUS,Ford,Focus,1,"2,000.00"
3,92928M178RUS,Ford,Focus,1,"8,594.59"
4,M302CH161RUS,Ford,Focus,1,500.00
...,...,...,...,...,...
195,72457Y96RUS,Ford,Focus,1,900.00
196,98047H178RUS,Ford,Focus,1,500.00
197,K559KC161RUS,Ford,Focus,1,100.00
198,X320MK197RUS,Ford,Focus,1,"7,500.00"


In [7]:
concat_rows = pd.concat([df, df_sample])

In [8]:
concat_rows

,CarNumber,Refund,Fines,Make,Model
0,Y163O8161RUS,2,"3,200.00",Ford,Focus
1,E432XX77RUS,1,"6,500.00",Toyota,Camry
2,7184TT36RUS,1,"2,100.00",Ford,Focus
3,X582HE161RUS,2,"2,000.00",Ford,Focus
4,92918M178RUS,1,"5,700.00",Ford,Focus
...,...,...,...,...,...
195,72457Y96RUS,1,900.00,Ford,Focus
196,98047H178RUS,1,500.00,Ford,Focus
197,K559KC161RUS,1,100.00,Ford,Focus
198,X320MK197RUS,1,"7,500.00",Ford,Focus


In [9]:
concat_rows.count()

CarNumber    925
Refund       925
Fines        925
Make         925
Model        916
dtype: int64

In [10]:
np.random.seed(21)
years = [np.random.randint(1980, 2020) for i in range(len(concat_rows))]
fines = concat_rows
fines['Year'] = years

In [11]:
fines

,CarNumber,Refund,Fines,Make,Model,Year
0,Y163O8161RUS,2,"3,200.00",Ford,Focus,1989
1,E432XX77RUS,1,"6,500.00",Toyota,Camry,1995
2,7184TT36RUS,1,"2,100.00",Ford,Focus,1984
3,X582HE161RUS,2,"2,000.00",Ford,Focus,2015
4,92918M178RUS,1,"5,700.00",Ford,Focus,2014
...,...,...,...,...,...,...
195,72457Y96RUS,1,900.00,Ford,Focus,1981
196,98047H178RUS,1,500.00,Ford,Focus,1992
197,K559KC161RUS,1,100.00,Ford,Focus,2007
198,X320MK197RUS,1,"7,500.00",Ford,Focus,2005


In [13]:
url = 'https://api.census.gov/data/2010/surname?get=NAME,COUNT&RANK=1:100'
res = requests.get(url).text

ConnectionError: HTTPSConnectionPool(host='api.census.gov', port=443): Max retries exceeded with url: /data/2010/surname?get=NAME,COUNT&RANK=1:100 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000252D1CB9F88>: Failed to establish a new connection: [WinError 10060] Попытка установить соединение была безуспешной, т.к. от другого компьютера за требуемое время не получен нужный отклик, или было разорвано уже установленное соединение из-за неверного отклика уже подключенного компьютера'))